In [1]:
import pandas as pd
import numpy as np
from functools import reduce
csv_path = 'Resources/purchase_data.csv'
purchase = pd.read_csv(csv_path)
purchase2 = pd.read_csv(csv_path)

In [2]:
first_answer = pd.DataFrame({'Total Players': [len(purchase['SN'].unique())]})
first_answer.head()

,Total Players
0,576


In [3]:
num_unique = len(purchase['Item ID'].unique())
revenue = purchase['Price'].sum()
total_num = purchase['Purchase ID'].count()
second_answer = pd.DataFrame({'Number of Unique Items': [num_unique], 
                              'Average Price': [revenue / total_num],
                              'Number of Purchases': [total_num],
                              'Total Revenue': [revenue]})
second_answer['Average Price'] = second_answer['Average Price'].map("${:.2f}".format)
second_answer['Total Revenue'] = second_answer['Total Revenue'].map("${:,.2f}".format)
second_answer

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,"$2,379.77"


In [4]:
unique_player = purchase2.drop_duplicates("SN")
total_unique = unique_player['Gender'].value_counts()
perc_unique = unique_player['Gender'].value_counts(normalize = True).multiply(100)
merge_1 = perc_unique.rename_axis('Gender').reset_index(name = 'Percentage of Players')
merge_2 = total_unique.rename_axis('Gender').reset_index(name = 'Total Count')
third_answer = pd.merge(merge_2, merge_1, on = "Gender")
third_answer['Percentage of Players'] = third_answer['Percentage of Players'].map("{:.2f}".format)
third_answer = third_answer.set_index("Gender")
third_answer

,Total Count,Percentage of Players
Gender,,
Male,484,84.03
Female,81,14.06
Other / Non-Disclosed,11,1.91


In [5]:
total_count = purchase['Gender'].value_counts()
gender_group = purchase.groupby(['Gender'])
price_total = gender_group['Price'].sum()
avg_price = price_total.divide(total_count)
avg_person = price_total.divide(total_unique)

merge_total = total_count.rename_axis('Gender').reset_index(name = 'Purchase Count')
merge_price_total = price_total.rename_axis('Gender').reset_index(name = 'Total Purchase Value')
merge_avg_price = avg_price.rename_axis('Gender').reset_index(name = 'Average Purchase Price')
merge_avg_person = avg_person.rename_axis('Gender').reset_index(name = 'Avg Total Purchase per Person')
merge_list = [merge_total, merge_avg_price, merge_price_total, merge_avg_person]
fourth_answer = reduce(lambda left,right: pd.merge(left,right,on='Gender'), merge_list)
for i in range(1,4):
    name = merge_list[i].columns[1]
    fourth_answer[name] = fourth_answer[name].map("${:,.2f}".format)
fourth_answer = fourth_answer.set_index("Gender")
fourth_answer

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Male,652,$3.02,"$1,967.64",$4.07
Female,113,$3.20,$361.94,$4.47
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [6]:
max_age = purchase["Age"].max()
bins = [0,9,14,19,24,29,34,39,max_age+1]
group_name = ['<10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40+']
unique_copy = unique_player.copy()
unique_copy['binning by age'] = pd.cut(unique_copy['Age'], bins, labels = group_name)
binning_total = unique_copy['binning by age'].value_counts(sort = False)
binning_perc = unique_copy['binning by age'].value_counts(normalize = True, sort = False).multiply(100)
merge_binning_total = binning_total.rename_axis(' ').reset_index(name = 'Total Count')
merge_binning_perc = binning_perc.rename_axis(' ').reset_index(name = 'Percentage of Players')
fifth_answer = pd.merge(merge_binning_total, merge_binning_perc, on = " ")
fifth_answer['Percentage of Players'] = fifth_answer['Percentage of Players'].map("{:.2f}".format)
fifth_answer = fifth_answer.set_index(" ")
fifth_answer

,Total Count,Percentage of Players
,,
<10,17,2.95
10-14,22,3.82
15-19,107,18.58
20-24,258,44.79
25-29,77,13.37
30-34,52,9.03
35-39,31,5.38
40+,12,2.08


In [7]:
purchase['binning by age'] = pd.cut(purchase['Age'], bins, labels = group_name)
binning_con_total = purchase['binning by age'].value_counts(sort = False)
binning_group = purchase.groupby('binning by age')
binning_price_total = binning_group['Price'].sum(sort = False)
binning_price_avg = binning_price_total.divide(binning_con_total)
binning_price_avg_person = binning_price_total.divide(binning_total)
merge_binning_con_total = binning_con_total.rename_axis(' ').reset_index(name = 'Purchase Count')
merge_binning_price_total = binning_price_total.rename_axis(' ').reset_index(name = 'Total Purchase Value')
merge_binning_price_avg = binning_price_avg.rename_axis(' ').reset_index(name = 'Average Purchase Price')
merge_binning_price_avg_person = binning_price_avg_person.rename_axis(' ').reset_index(name = 'Avg Total Purchase per Person')
merge_binning_list = [merge_binning_con_total, merge_binning_price_avg, 
                      merge_binning_price_total, merge_binning_price_avg_person]
sixth_answer = reduce(lambda left,right: pd.merge(left,right,on=' '), merge_binning_list)
for i in range(1,4):
    name = merge_binning_list[i].columns[1]
    sixth_answer[name] = sixth_answer[name].map("${:,.2f}".format)
sixth_answer = sixth_answer.set_index(" ")
sixth_answer

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


In [8]:
sn_group = purchase.groupby('SN')
price_sum = sn_group['Price'].sum()
sn_count = sn_group["Price"].count()
sn_avg = sn_group['Price'].mean()
merge_sn_count = sn_count.rename_axis('SN').reset_index(name = 'Purchase Count')
merge_sn_avg = sn_avg.rename_axis('SN').reset_index(name = 'Average Purchase Price')
merge_price_sum = price_sum.rename_axis('SN').reset_index(name = 'Total Purchase Value')
merge_sn_1 = pd.merge(merge_sn_count, merge_sn_avg, on = 'SN')
seventh_answer = pd.merge(merge_sn_1, merge_price_sum, on = 'SN')
seventh_answer = seventh_answer.sort_values('Total Purchase Value', ascending = False)
seventh_answer["Average Purchase Price"] = seventh_answer["Average Purchase Price"].map("${:,.2f}".format)
seventh_answer["Total Purchase Value"] = seventh_answer['Total Purchase Value'].map("${:,.2f}".format)
seventh_answer = seventh_answer.set_index("SN")
seventh_answer.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [9]:
item_group = purchase.groupby('Item ID')
purchase_count = item_group['SN'].count()
item_price = item_group['Price'].apply(lambda x: x)
item_name = item_group['Item Name'].apply(lambda x: x)
item_total = item_group['Price'].sum()
merge_purchase_count = purchase_count.rename_axis('Item ID').reset_index(name = 'Purchase Count')
merge_item_price = item_price.rename_axis('Item ID').reset_index(name = 'Item Price')
merge_item_name = item_name.rename_axis('Item ID').reset_index(name = 'Item Name')
merge_item_total = item_total.rename_axis('Item ID').reset_index(name = 'Total Purchase Value')
item_list = [merge_item_name, merge_purchase_count, merge_item_price, merge_item_total]
eighth_answer = reduce(lambda left,right: pd.merge(left,right,on='Item ID'), item_list)
eighth_answer_copy = eighth_answer.copy()
for i in range(2,4):
    name = item_list[i].columns[1]
    eighth_answer[name] = eighth_answer[name].map("${:,.2f}".format)
eighth_answer = eighth_answer.sort_values('Purchase Count', ascending = False)
eighth_answer = eighth_answer.set_index('Item ID')
eighth_answer.head()

,Item Name,Purchase Count,Item Price,Total Purchase Value
Item ID,,,,
178,"Despair, Favor of Due Diligence",12,$4.60,$50.76
145,Hopeless Ebon Dualblade,9,$1.33,$41.22
108,Malificent Bag,9,$1.75,$31.77
82,Azurewrath,9,$4.40,$44.10
19,"Blazefury, Protector of Delusions",8,$4.64,$8.16


In [10]:
nineth_answer = eighth_answer_copy.sort_values("Total Purchase Value", ascending = False)
for i in range(2,4):
    name = item_list[i].columns[1]
    nineth_answer[name] = nineth_answer[name].map("${:,.2f}".format)
nineth_answer = nineth_answer.set_index('Item ID')
nineth_answer.head()

,Item Name,Purchase Count,Item Price,Total Purchase Value
Item ID,,,,
178,"Despair, Favor of Due Diligence",12,$4.60,$50.76
82,Azurewrath,9,$4.40,$44.10
145,Hopeless Ebon Dualblade,9,$1.33,$41.22
92,"Betrayal, Whisper of Grieving Widows",8,$3.94,$39.04
103,"Thorn, Satchel of Dark Souls",8,$1.33,$34.80
